In [54]:
import mesa
import networkx as nx
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error

In [55]:
cities = pd.read_csv("data/cities.csv")
demographic = pd.read_csv("data/demographic.csv")
turnout = pd.read_csv("data/turnout.csv")

In [56]:
print(type(cities))
print(cities.head())

<class 'pandas.core.frame.DataFrame'>
          city   latitude   longitude
0       manila  14.590449  120.980362
1  quezon city  14.651055  121.048625
2     caloocan  14.651348  120.972400
3       makati  14.556795  121.021123
4     marikina  14.633108  121.099354


In [57]:
threshold = 20 # Connect cities within this distance from each other

G = nx.Graph()
for _, row in cities.iterrows():
    G.add_node(row["city"], pos=(row["latitude"], row["longitude"]))

for city1 in G.nodes:
    for city2 in G.nodes:
        if city1 == city2:
            continue
        dist = np.linalg.norm(np.array(G.nodes[city1]["pos"]) - np.array(G.nodes[city2]["pos"]))

        if dist < threshold:
            print(f"Connected {city1} and {city2}")
            G.add_edge(city1, city2, weight=1/dist)

Connected manila and quezon city
Connected manila and caloocan
Connected manila and makati
Connected manila and marikina
Connected manila and parañaque
Connected manila and pasig
Connected manila and valenzuela
Connected manila and pasay
Connected manila and las piñas
Connected manila and mandaluyong
Connected manila and muntinlupa
Connected manila and malabon
Connected manila and navotas
Connected manila and san juan
Connected manila and taguig
Connected manila and pateros
Connected quezon city and manila
Connected quezon city and caloocan
Connected quezon city and makati
Connected quezon city and marikina
Connected quezon city and parañaque
Connected quezon city and pasig
Connected quezon city and valenzuela
Connected quezon city and pasay
Connected quezon city and las piñas
Connected quezon city and mandaluyong
Connected quezon city and muntinlupa
Connected quezon city and malabon
Connected quezon city and navotas
Connected quezon city and san juan
Connected quezon city and taguig
C

In [58]:
age_weight = {'0-4': 0, '5-9': 0, '10-14': 0, '15-19': 0.2, '20-24': 0.7, '25-29': 0.8, '30-34': 0.85, '35-39': 0.9, '40-44': 0.9, '45-49': 0.9, '50-54': 0.95, '55-59': 0.95, '60-64': 0.95, '65-69': 0.9, '70-74': 0.85, '75-79': 0.8, '80+': 0.7}
sex_weight = {'female': 1.1, 'male': 1.0}

self_weight = 0.7
same_city_weight = 0.15
nearby_weight = 0.15

class Agent(mesa.Agent):
    def __init__(self, id, model, city, age_group, sex):
        super().__init__(id, model)
        self.city = city
        self.age_group = age_group
        self.sex = sex
        city_turnout = turnout[turnout["city"] == city]
        self.turnout_prob = city_turnout["voted"].values[0] / city_turnout["registered"].values[0]

        self.turnout_prob *= self.demographic_adjustment()
    
    def demographic_adjustment(self):
        return age_weight[self.age_group] * sex_weight[self.sex]

    def step(self):
        neighbors = self.model.grid.get_cell_list_contents([self.city])
        same_city_avg = np.mean([a.turnout_prob for a in neighbors if a != self])

        nearby_agents = []
        for neighbor_city in G.neighbors(self.city):
            nearby_agents += self.model.grid.get_cell_list_contents([neighbor_city])
        
        nearby_avg = np.mean([a.turnout_prob for a in nearby_agents]) if nearby_agents else self.turnout_prob

        self.turnout_prob = (
            self_weight * self.turnout_prob + 
            same_city_weight * same_city_avg +
            nearby_weight * nearby_avg
        )


In [59]:
scale = 100
class Model(mesa.Model):
    def __init__(self):
        self.schedule = mesa.time.RandomActivation(self)
        self.grid = mesa.space.NetworkGrid(G)
        self.agent_id = 0


        for _, row in demographic.iterrows():
            city = row["city"]
            age_group = row["age_group"]
            sex = row["sex"]
            count = int(row["population"])

            for _ in range(int(count / scale)):
                agent = Agent(self.agent_id, self, city, age_group, sex)
                self.schedule.add(agent)
                self.grid.place_agent(agent, city)
                self.agent_id += 1
    def step(self):
        self.schedule.step()

In [ ]:
model = Model()
for i in range(10):
    print(f"Step {i+1}")
    model.step()

c:\Users\beaux\anaconda3\envs\consunji_beaux\lib\site-packages\mesa\agent.py:52: FutureWarning: The Mesa Model class was not initialized. In the future, you need to explicitly initialize the Model by calling super().__init__() on initialization.
  self.model.register_agent(self)


Step 1


In [ ]:
simulated_turnout = {}
for agent in model.schedule.agents:
    if agent.city not in simulated_turnout:
        simulated_turnout[agent.city] = []
    simulated_turnout[agent.city].append(agent.turnout_prob)

simulated_turnout_avg = {
    city: np.mean(probs) * turnout[turnout["city"] == city]["registered_voters"].values[0]
    for city, probs in simulated_turnout.items()
}

actual_turnout = turnout.set_index("city")["actual_voters"].to_dict()
mse = mean_squared_error(list(actual_turnout.values()), list(simulated_turnout_avg.values()))
print(f"Mean Squared Error: {mse: .2f}")